In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
#tf.keras.wrappers.skikit_learn.KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Set random seed
np.random.seed(0)

In [ ]:
data = pd.read_csv(r'D:\MSDS-Capstone-Project\implementation\12Z.csv')
data['Date'] = pd.to_datetime(data['Date'])
data = data.sort_values(by='Date')
data.set_index('Date', inplace=True)
del data['lifted index (500-100mb)']

In [ ]:
data.head()

In [ ]:
tor_count = pd.read_csv(r'D:\MSDS-Capstone-Project\implementation\tornadoCounts.csv', index_col=1)
tor_count = tor_count.loc[:, ~tor_count.columns.str.contains('^Unnamed')]
tor_count.index = pd.to_datetime(tor_count.index)
#tor_count['date'] = pd.to_datetime(tor_count['date'])
#tor_count = tor_count.sort_values(by='date')
#tor_count.set_index('date', inplace=True)

In [ ]:
tor_count.head()

In [ ]:
tor_count.rename(columns={'count':'COUNT'}, inplace=True)
tor_count.loc[tor_count.COUNT < 5, 'countBinary'] = 0
tor_count.loc[tor_count.COUNT >= 5, 'countBinary'] = 1
tor_count.countBinary = tor_count.countBinary.astype(int)
tor_count.tail(20)

In [ ]:
merge=pd.merge(data,tor_count, how='inner', left_index=True, right_index=True)
merge = merge.sample(frac=1)
merge.tail(20)

In [ ]:
merge.head()

In [ ]:
merge.tail()

In [ ]:
tor_count.tail()

In [ ]:
#sns.pairplot(merge, hue='Count')

In [ ]:
#sns.heatmap(merge.corr(), annot=True)

In [ ]:
# creating input featues and target variables
X = merge.iloc[:,0:26]
y = merge.iloc[:,27]

In [ ]:
X.tail(20)

In [ ]:
y.tail(20)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
scaler = StandardScaler().fit(X_train)
X_Train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
model2 = keras.Sequential()
model2.add(keras.layers.Dense(26, activation='relu', input_shape=(26,)))
model2.add(keras.layers.Dense(23, activation='relu'))
model2.add(keras.layers.Dense(20, activation='relu'))
#model.add(keras.layers.Dropout(0.2))
#model.add(keras.layers.Dense(17, activation='relu'))
#model.add(keras.layers.Dense(14, activation='relu'))
#model.add(keras.layers.Dropout(0.2))
#model.add(keras.layers.Dense(13, activation='relu'))
#model.add(keras.layers.Dense(10, activation='relu'))
#model.add(keras.layers.Dropout(0.2))
#model.add(keras.layers.Dense(7, activation='relu'))
#model.add(keras.layers.Dense(4, activation='relu'))
#model.add(keras.layers.Dropout(0.2))
model2.add(keras.layers.Dense(1, activation='sigmoid'))
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model2.fit(X_train, y_train, validation_split=0.1, epochs=40, batch_size=10)
eval_model=model2.evaluate(X_train, y_train)
print(eval_model)
y_pred = model2.predict(X_test)
y_pred =(y_pred>0.5)
cm = confusion_matrix(y_test, y_pred)
print(cm)


#score = model2.evaluate(X_test, y_test,verbose=1)
#print(score)

In [ ]:
model.save(r'D:\MSDS-Capstone-Project\implementation\savedModels\12ZinitialModel.h5')

In [ ]:
# Tune the batch size and number of epochs
def create_model():
    model = keras.Sequential()
    model.add(keras.layers.Dense(26, activation='relu', input_shape=(26,)))
    model.add(keras.layers.Dense(23, activation='relu'))
    model.add(keras.layers.Dense(20, activation='relu'))
    #model.add(keras.layers.Dropout(0.2))
    #model.add(keras.layers.Dense(17, activation='relu'))
    #model.add(keras.layers.Dense(14, activation='relu'))
    #model.add(keras.layers.Dropout(0.2))
    #model.add(keras.layers.Dense(13, activation='relu'))
    #model.add(keras.layers.Dense(10, activation='relu'))
    #model.add(keras.layers.Dropout(0.2))
    #model.add(keras.layers.Dense(7, activation='relu'))
    #model.add(keras.layers.Dense(4, activation='relu'))
    #model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, verbose=0)

batch_size = [20, 40, 60, 80, 100]
epochs = [20, 40, 60, 80, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# use the best batch and epochs found by hypertuning
model3 = keras.Sequential()
model3.add(keras.layers.Dense(26, activation='relu', input_shape=(26,)))
model3.add(keras.layers.Dense(23, activation='relu'))
model3.add(keras.layers.Dense(20, activation='relu'))
model3.add(keras.layers.Dense(1, activation='sigmoid'))
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model3.fit(X_train, y_train, validation_split=0.1, epochs=100, batch_size=40)
eval_model=model3.evaluate(X_train, y_train)
print(eval_model)
y_pred = model3.predict(X_test)
y_pred =(y_pred>0.5)
cm = confusion_matrix(y_test, y_pred)
print(cm)

In [ ]:
def create_model(optimizer='adam'):
    model = keras.Sequential()
    model.add(keras.layers.Dense(26, activation='relu', input_shape=(26,)))
    model.add(keras.layers.Dense(23, activation='relu'))
    model.add(keras.layers.Dense(20, activation='relu'))
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, epochs=100, batch_size=40, verbose=0)

optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))



In [ ]:
# use the best batch and epochs found by hypertuning and the Adamax optimizer
model4 = keras.Sequential()
model4.add(keras.layers.Dense(26, activation='relu', input_shape=(26,)))
model4.add(keras.layers.Dense(23, activation='relu'))
model4.add(keras.layers.Dense(20, activation='relu'))
model4.add(keras.layers.Dense(1, activation='sigmoid'))
model4.compile(loss='binary_crossentropy', optimizer='Adamax', metrics=['accuracy'])

model4.fit(X_train, y_train, epochs=100, batch_size=40)
eval_model=model4.evaluate(X_train, y_train)
print(eval_model)
y_pred = model4.predict(X_test)
y_pred =(y_pred>0.5)
cm = confusion_matrix(y_test, y_pred)
print(cm)

In [ ]:
# tune the learning rate and beta
def create_model(learn_rate=0.01, beta_1=0.0, beta_2=0.0):
    model = keras.Sequential()
    model.add(keras.layers.Dense(26, activation='relu', input_shape=(26,)))
    model.add(keras.layers.Dense(23, activation='relu'))
    model.add(keras.layers.Dense(20, activation='relu'))
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    optimizer = tf.optimizers.Adamax(lr=learn_rate, beta_1 = beta_1, beta_2 = beta_2)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

model = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, epochs=100, batch_size=40, verbose=0)

learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
beta_1 = [0.0, 0.001, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
beta_2 = [0.0, 0.001, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
param_grid = dict(learn_rate=learn_rate, beta_1=beta_1, beta_2=beta_2)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


In [ ]:
#epochs = 60
#learning_rate = 0.1
#decay_rate = learning_rate / epochs
#momentum = 0.8
#opt = keras.optimizers.SGD(lr=learning_rate, momentum=momentum, decay=decay_rate)
#model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

def single_hidden_model(neurons=1):
    model = keras.Sequential()
    model.add(keras.layers.Dense(neurons, input_dim=26, kernel_initializer='uniform', activation='relu'))
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(1, kernel_initializer='uniform', activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=single_hidden_model, epochs=100, batch_size=20, verbose=0)
neurons = [1, 5, 10, 15, 20, 25, 26, 30, 34, 40, 45, 50, 55, 60, 65]
param_grid = dict(neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=6, cv=3)
grid_result = grid.fit(X_train, y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Tune the training optimization algotrithm
#model = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
#optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
#param_grid = dict(optimizer=optimizer)
#grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=6, cv=3)
#grid_result = grid.fit(X_train,y_train)
#print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
#means = grid_result.cv_results_['mean_test_score']
#stds = grid_result.cv_results_['std_test_score']
#params = grid_result.cv_results_['params']
#for mean, stdev, param in zip(means, stds, params):
#    print("%f (%f) with: %r" % (mean, stdev, param))



#model = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model)
#batch_size = [10, 20, 40, 60, 80, 100]
#epochs = [10, 50, 100]
#optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
#param_grid = dict(batch_size=batch_size, epochs=epochs, optimizer=optimizer)
#grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=6, cv=3)
#grid_result = grid.fit(X, y)

#print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
#means = grid_result.cv_results_['mean_test_score']
#stds = grid_result.cv_results_['std_test_score']
#params = grid_result.cv_results_['params']
#for mean, stdev, param in zip(means, stds, params):
#    print("%f (%f) with: %r" % (mean, stdev, param))
